# Data Cleaning
I will use this notebook to clean the data set, trying to replace nulls in as many cases where it makes logical and mathematical sense. I will roll this into a 'clean_data' function so I can quickly clean the data rather than having to re run all the code chunks each time. I will then split the data into test and train csv files.

## 1 Imports

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
from sklearn.metrics import confusion_matrix
from sklearn import datasets, linear_model
import matplotlib.pyplot as plt
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import scipy

plt.rcParams["figure.figsize"] = (20,10)

In [3]:
# Use external source to map location to latitude
latdict = {
    "Adelaide": "-34.91490432431718",
    "Albany": "-35.02474905886614",
    "Albury": "-36.06534287815004",
    "AliceSprings": "-23.69805670084839",
    "BadgerysCreek": "-33.87695470721896",
    "Ballarat": "-37.561716309375115",
    "Bendigo": "-36.7554908916319",
    "Brisbane": "-27.475717897882603",
    "Cairns": "-16.92781685482217",
    "Canberra": "-35.30773503404086",
    "Cobar": "-31.504167276363866",
    "CoffsHarbour": "-30.298002978449876",
    "Dartmoor": "-37.925451468193096",
    "Darwin": "-12.464083602807655",
    "GoldCoast": "-28.014572596815473",
    "Hobart": "-42.88435139189265",
    "Katherine": "-14.456998531987214",
    "Launceston": "-41.43266508391751",
    "Melbourne": "-37.832462412802045",
    "MelbourneAirport": "-37.66879996801125",
    "Mildura": "-34.20248688142015",
    "Moree": "-29.428569880868324",
    "MountGambier": "-37.82374072116639",
    "MountGinini": "-35.53217670083329",
    "Newcastle": "-32.931643653584594",
    "Nhil": "-36.322637084347",
    "NorahHead": "-33.28147041348496",
    "NorfolkIsland": "-29.034135426151366",
    "Nuriootpa": "-34.468017870418535",
    "PearceRAAF": "-31.667583781229556",
    "Penrith": "-33.744842332442595",
    "Perth": "-31.986585351851893",
    "PerthAirport": "-31.93846563915158",
    "Portland": "-38.34832717511207",
    "Richmond": "-37.818604503111914",
    "Sale": "-38.100948826120415",
    "SalmonGums": "-32.842740170834155",
    "Sydney": "-33.86371676520854",
    "SydneyAirport": "-33.9421060860826",
    "Townsville": "-19.264341792547878",
    "Tuggeranong": "-35.46185196956041",
    "Uluru": "-25.346664599700304",
    "WaggaWagga": "-35.11756402381749",
    "Walpole": "-34.966482092339895",
    "Watsonia": "-37.70991777367249",
    "Williamtown": "-37.86099076195903",
    "Witchcliffe": "-34.030314070821824",
    "Wollongong": "-34.43196249792971",
    "Woomera": "-31.18123956846809"
}

In [4]:
# Use external source to map location to longitude
longdict = {
    "Adelaide": "138.59602633990767",
    "Albany": "117.88316237554488",
    "Albury": "146.93525279045156",
    "AliceSprings": "133.8787600142725",
    "BadgerysCreek": "150.7484363529729",
    "Ballarat": "143.84930592526567",
    "Bendigo": "144.2801518461566",
    "Brisbane": "153.02887201830356",
    "Cairns": "145.7430348255027",
    "Canberra": "149.1229092502184",
    "Cobar": "145.83821542838913",
    "CoffsHarbour": "153.12107860997904",
    "Dartmoor": "141.27285789793294",
    "Darwin": "130.84334958238955",
    "GoldCoast": "153.40503715057645",
    "Hobart": "147.3286420858417",
    "Katherine": "132.26809234822562",
    "Launceston": "147.1457335103628",
    "Melbourne": "144.96053444028217",
    "MelbourneAirport": "144.84076980637124",
    "Mildura": "142.12879595976347",
    "Moree": "149.7771468517488",
    "MountGambier": "140.78081550124955",
    "MountGinini": "148.77330846278252",
    "Newcastle": "151.7590444755576",
    "Nhil": "141.64167651316964",
    "NorahHead": "151.57743077776965",
    "NorfolkIsland": "167.9549498768209",
    "Nuriootpa": "138.9928831519288",
    "PearceRAAF": "116.02918925414023",
    "Penrith": "150.68695749702303",
    "Perth": "115.8515962142354",
    "PerthAirport": "115.96748523272586",
    "Portland": "141.60745855632877",
    "Richmond": "145.00019734623154",
    "Sale": "147.08029490271954",
    "SalmonGums": "121.64337708357834",
    "Sydney": "151.20782045617366",
    "SydneyAirport": "151.1763664478918",
    "Townsville": "146.81702724273325",
    "Tuggeranong": "149.1116317085056",
    "Uluru": "131.03645908713938",
    "WaggaWagga": "147.3544221110696",
    "Walpole": "116.75417748943707",
    "Watsonia": "145.0859484467225",
    "Williamtown": "144.89555627293157",
    "Witchcliffe": "115.10127681861184",
    "Wollongong": "150.88773768877098",
    "Woomera": "136.81002833379935"
}

## 2 Initial Inspection and Column Additions

In [5]:
# Load the data from csv
df = pd.read_csv("Kaggle Training Data.csv")
df.isnull().sum()

Date                 0
Location             0
MinTemp           1087
MaxTemp            882
Rainfall          2607
Evaporation      45945
Sunshine         50109
WindGustDir       8548
WindGustSpeed     8485
WindDir9am        8872
WindDir3pm        2818
WindSpeed9am      1568
WindSpeed3pm      1764
Humidity9am       2117
Humidity3pm       2333
Pressure9am      11876
Pressure3pm      11842
Cloud9am         43137
Cloud3pm         44450
Temp9am           1444
Temp3pm           1620
RainToday         2607
RainTomorrow      2607
dtype: int64

In [6]:
# Convert dates to datetimes
df['Date'] = pd.to_datetime(df['Date'], format="%d/%m/%Y")

In [7]:
# Convert location to lat/long and ensure it's float not object
df['Latitude'] = df['Location'].map(latdict)
df['Latitude'] = df['Latitude'].astype(float)

df['Longitude'] = df['Location'].map(longdict)
df['Longitude'] = df['Longitude'].astype(float)

In [8]:
# Add some other useful date values ( will one-hot encode these later on)
df['Month_Num'] = pd.DatetimeIndex(df['Date']).month
df['Year'] = pd.DatetimeIndex(df['Date']).year
df['DayOfYear'] = pd.DatetimeIndex(df['Date']).dayofyear

## 3 Cleaning

### 3.1 Using Linear Regression between two strongly correlated variables to fill missing cloud, sunshine and evaporation values

In [9]:
# Linear Regression Models for extrapolating 'Cloud9am' from cloud, humidity and sunshine features

# Extrapolating Cloud9am from Humidity 3pm
x1 = df.dropna()[['Humidity3pm']].values
y1 = df.dropna()[['Cloud9am']].values
hum3_cloud9 = linear_model.LinearRegression()
hum3_cloud9.fit(x1, y1)
print('Gradient = ' + str(hum3_cloud9.coef_))
print('Intercept = ' + str(hum3_cloud9.intercept_))

# Extrapolating Cloud9am from Sunshine
x7 = df.dropna()[['Sunshine']].values
y7 = df.dropna()[['Cloud9am']].values
sunshine_cloud9 = linear_model.LinearRegression()
sunshine_cloud9.fit(x7, y7)
print('Gradient = ' + str(sunshine_cloud9.coef_))
print('Intercept = ' + str(sunshine_cloud9.intercept_))

# Extrapolating Cloud9am from Cloud3
x10 = df.dropna()[['Cloud3pm']].values
y10 = df.dropna()[['Cloud9am']].values
cloud3_cloud9 = linear_model.LinearRegression()
cloud3_cloud9.fit(x10, y10)
print('Gradient = ' + str(cloud3_cloud9.coef_))
print('Intercept = ' + str(cloud3_cloud9.intercept_))

Gradient = [[0.07100456]]
Intercept = [0.7103748]
Gradient = [[-0.50787855]]
Intercept = [8.15432275]
Gradient = [[0.65498279]]
Intercept = [1.3981431]


In [10]:
# Linear Regression Models for extrapolating 'Cloud3pm' from cloud, humidity and sunshine features

# Extrapolating Cloud3pm from Humidity3pm
x2 = df.dropna()[['Humidity3pm']].values
y2 = df.dropna()[['Cloud3pm']].values
hum3_cloud3 = linear_model.LinearRegression()
hum3_cloud3.fit(x2, y2)
print('Gradient = ' + str(hum3_cloud3.coef_))
print('Intercept = ' + str(hum3_cloud3.intercept_))

# Extrapolating Cloud3pm from Humidity9am
x13 = df.dropna()[['Humidity9am']].values
y13 = df.dropna()[['Cloud3pm']].values
hum9_cloud3 = linear_model.LinearRegression()
hum9_cloud3.fit(x13, y13)
print('Gradient = ' + str(hum9_cloud3.coef_))
print('Intercept = ' + str(hum9_cloud3.intercept_))

# Extrapolating Cloud3pm from Sunshine
x8 = df.dropna()[['Sunshine']].values
y8 = df.dropna()[['Cloud3pm']].values
sunshine_cloud3 = linear_model.LinearRegression()
sunshine_cloud3.fit(x8, y8)
print('Gradient = ' + str(sunshine_cloud3.coef_))
print('Intercept = ' + str(sunshine_cloud3.intercept_))

# Extrapolating Cloud3pm from Cloud9am
x9 = df.dropna()[['Cloud9am']].values
y9 = df.dropna()[['Cloud3pm']].values
cloud9_cloud3 = linear_model.LinearRegression()
cloud9_cloud3.fit(x9, y9)
print('Gradient = ' + str(cloud9_cloud3.coef_))
print('Intercept = ' + str(cloud9_cloud3.intercept_))

Gradient = [[0.06738554]]
Intercept = [0.98621665]
Gradient = [[0.05049076]]
Intercept = [0.99754372]
Gradient = [[-0.49414921]]
Intercept = [8.14456851]
Gradient = [[0.5848948]]
Intercept = [1.85416821]


In [11]:
# Linear Regression Models for extrapolating 'Sunshine' from cloud, humidty and temp features

# Extrapolating Sunshine from Cloud9am
x3 = df.dropna()[['Cloud9am']].values
y3 = df.dropna()[['Sunshine']].values
cloud9_sunshine = linear_model.LinearRegression()
cloud9_sunshine.fit(x3, y3)
print('Gradient = ' + str(cloud9_sunshine.coef_))
print('Intercept = ' + str(cloud9_sunshine.intercept_))

# Extrapolating Sunshine from Cloud3pm
x4 = df.dropna()[['Cloud3pm']].values
y4 = df.dropna()[['Sunshine']].values
cloud3_sunshine = linear_model.LinearRegression()
cloud3_sunshine.fit(x4, y4)
print('Gradient = ' + str(cloud3_sunshine.coef_))
print('Intercept = ' + str(cloud3_sunshine.intercept_))

# Extrapolating Sunshine from Humidity3pm
x5 = df.dropna()[['Humidity3pm']].values
y5 = df.dropna()[['Sunshine']].values
hum3_sunshine = linear_model.LinearRegression()
hum3_sunshine.fit(x5, y5)
print('Gradient = ' + str(hum3_sunshine.coef_))
print('Intercept = ' + str(hum3_sunshine.intercept_))

# Extrapolating Sunshine from Humidity9amm
x11 = df.dropna()[['Humidity9am']].values
y11 = df.dropna()[['Sunshine']].values
hum9_sunshine = linear_model.LinearRegression()
hum9_sunshine.fit(x11, y11)
print('Gradient = ' + str(hum9_sunshine.coef_))
print('Intercept = ' + str(hum9_sunshine.intercept_))

# Extrapolating Sunshine from Temp3pm
x12 = df.dropna()[['Temp3pm']].values
y12 = df.dropna()[['Sunshine']].values
temp3_sunshine = linear_model.LinearRegression()
temp3_sunshine.fit(x12, y12)
print('Gradient = ' + str(temp3_sunshine.coef_))
print('Intercept = ' + str(temp3_sunshine.intercept_))

Gradient = [[-0.91186285]]
Intercept = [11.57878106]
Gradient = [[-0.99352747]]
Intercept = [12.02020415]
Gradient = [[-0.11671639]]
Intercept = [13.5107794]
Gradient = [[-0.10124569]]
Intercept = [14.40179237]
Gradient = [[0.26731699]]
Intercept = [1.69084112]


In [12]:
# Linear Regression Models for Extrapolating 'Evaporation' from 'Maxtemp'

# Extrapolating Evaporation from MaxTemp
x6 = df.dropna()[['MaxTemp']].values
y6 = df.dropna()[['Evaporation']].values
maxtemp_evap = linear_model.LinearRegression()
maxtemp_evap.fit(x6, y6)
print('Gradient = ' + str(maxtemp_evap.coef_))
print('Intercept = ' + str(maxtemp_evap.intercept_))

Gradient = [[0.34620242]]
Intercept = [-2.91298131]


In [14]:
# Replaces 'Cloud9am' nulls with linearly regressed sunshine or cloud3pm of humidity3pm values 
cloud9am = np.array(df['Cloud9am']).reshape(-1, 1) 
sunshine = np.array(df['Sunshine']).reshape(-1, 1)
cloud3pm = np.array(df['Cloud3pm']).reshape(-1, 1)
humidity3pm = np.array(df['Humidity3pm']).reshape(-1, 1)
for x in range(0, len(cloud9am)):
    if np.isnan(cloud9am[x]):
        if not np.isnan(sunshine[x]):
            cloud9am[x] = sunshine_cloud9.predict(sunshine[x].reshape(-1,1))
        else: 
            if not np.isnan(cloud3pm[x]):
                cloud9am[x] = cloud3_cloud9.predict(cloud3pm[x].reshape(-1,1))
            else:
                if not np.isnan(humidity3pm[x]):
                    cloud9am[x] = hum3_cloud9.predict(humidity3pm[x].reshape(-1,1))
                      
cloud9am_cleaned = cloud9am
print('Total Entries = ' + str(len(cloud9am)))
print('Initial NaNs = 43137')
print('Remaining NaNs = ' + str(np.count_nonzero(np.isnan(cloud9am_cleaned))))

Total Entries = 116367
Initial NaNs = 43137
Remaining NaNs = 982


In [15]:
# Replaces 'Cloud3pm' nulls with linearly regressed sunshine or cloud9am or humidity3pm values 
cloud9am = np.array(df['Cloud9am']).reshape(-1, 1) 
sunshine = np.array(df['Sunshine']).reshape(-1, 1)
cloud3pm = np.array(df['Cloud3pm']).reshape(-1, 1)
humidity3pm = np.array(df['Humidity3pm']).reshape(-1, 1)
humidity9am = np.array(df['Humidity9am']).reshape(-1, 1)
for x in range(0, len(cloud3pm)):
    if np.isnan(cloud3pm[x]):
        if not np.isnan(sunshine[x]):
            cloud3pm[x] = sunshine_cloud3.predict(sunshine[x].reshape(-1,1))
        else: 
            if not np.isnan(cloud9am[x]):
                cloud3pm[x] = cloud9_cloud3.predict(cloud9am[x].reshape(-1,1))
            else:
                if not np.isnan(humidity3pm[x]):
                    cloud3pm[x] = hum3_cloud3.predict(humidity3pm[x].reshape(-1,1))
                else:
                    if not np.isnan(humidity9am[x]):
                        cloud9am[x] = hum9_cloud3.predict(humidity9am[x].reshape(-1,1))
                        
                    
cloud3pm_cleaned = cloud3pm
print('Total Entries = ' + str(len(cloud3pm)))
print('Initial NaNs = 44450')
print('Remaining NaNs = ' + str(np.count_nonzero(np.isnan(cloud3pm_cleaned))))

Total Entries = 116367
Initial NaNs = 44450
Remaining NaNs = 982


In [16]:
# Replaces 'Sunshine' nulls with linearly regressed cloud3pm or cloud9am or humidity3pm or humidity9am values 
cloud9am = np.array(df['Cloud9am']).reshape(-1, 1) 
sunshine = np.array(df['Sunshine']).reshape(-1, 1)
cloud3pm = np.array(df['Cloud3pm']).reshape(-1, 1)
humidity3pm = np.array(df['Humidity3pm']).reshape(-1, 1)
humidity9am = np.array(df['Humidity9am']).reshape(-1, 1)
temp3pm = np.array(df['Temp3pm']).reshape(-1, 1)
for x in range(0, len(sunshine)):
    if np.isnan(sunshine[x]):
        if not np.isnan(cloud3pm[x]):
            sunshine[x] = cloud3_sunshine.predict(cloud3pm[x].reshape(-1,1))
        else: 
            if not np.isnan(cloud9am[x]):
                sunshine[x] = cloud9_sunshine.predict(cloud9am[x].reshape(-1,1))
            else:
                if not np.isnan(humidity3pm[x]):
                    sunshine[x] = hum3_sunshine.predict(humidity3pm[x].reshape(-1,1))
                else:
                    if not np.isnan(humidity9am[x]):
                        sunshine[x] = hum9_sunshine.predict(humidity9am[x].reshape(-1,1))
                    else:
                        if not np.isnan(temp3pm[x]):
                            sunshine[x] = temp3_sunshine.predict(temp3pm[x].reshape(-1,1))
                        
sunshine_cleaned = sunshine
print('Total Entries = ' + str(len(sunshine)))
print('Initial NaNs = 50109')
print('Remaining NaNs = ' + str(np.count_nonzero(np.isnan(sunshine_cleaned))))

Total Entries = 116367
Initial NaNs = 50109
Remaining NaNs = 419


In [17]:
# Replaces 'Evaporation' Nulls with max temp
evaporation = np.array(df['Evaporation']).reshape(-1, 1)
maxtemp = np.array(df['MaxTemp']).reshape(-1, 1)
for x in range(0, len(evaporation)):
    if np.isnan(evaporation[x]):
        if not np.isnan(maxtemp[x]):
            evaporation[x] = maxtemp_evap.predict(maxtemp[x].reshape(-1,1))
        
evaporation_cleaned = evaporation
print('Total Entries = ' + str(len(evaporation)))
print('Initial NaNs = 45945')
print('Remaining NaNs = ' + str(np.count_nonzero(np.isnan(evaporation_cleaned))))

Total Entries = 116367
Initial NaNs = 45945
Remaining NaNs = 452


In [18]:
# Replaces columns in main df with cleaned versions
df['Cloud9am'] = cloud9am_cleaned
df['Cloud3pm'] = cloud3pm_cleaned
df['Sunshine'] = sunshine_cleaned
df['Evaporation'] = evaporation_cleaned

### 3.2 Windspeed and Pressure donated from correlated locations

In [22]:
# Take max of AM and PM for newwindgustspeed
windspeed_df = df[['Date','Location','WindGustSpeed','WindSpeed9am','WindSpeed3pm']]
windspeed_df['MaxSpeed'] = windspeed_df.max(axis=1)
df['WindGustSpeed'] = windspeed_df['MaxSpeed']

<ipython-input-22-58fa7ae8644b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  windspeed_df['MaxSpeed'] = windspeed_df.max(axis=1)


In [23]:
# Copy pressure values from defined correlated places
new_pressure_df = df[['Date','Location','Pressure9am','Pressure3pm']]

In [24]:
# pairs of correlated destinations
pairs = [['MountGinini','Canberra'],
          ['Newcastle','Williamtown'],
          ['Penrith','BadgerysCreek'],
          ['SalmonGums','Walpole']]

In [25]:
# replace missing pressure values from location pair
collect_df = pd.DataFrame(columns=new_pressure_df.columns)

for p in pairs:
    # define missing/donor pressure pairs
    df_missing = new_pressure_df[new_pressure_df['Location']==p[0]]
    df_donor = new_pressure_df[new_pressure_df['Location']==p[1]]
    df_missing.set_index("Date", inplace=True)
    df_donor.set_index("Date", inplace=True)
    
    #replaces missing values with donor values
    filled_df = df_missing.join(df_donor, rsuffix='_donor')
    filled_df['Pressure3pm'] = filled_df['Pressure3pm_donor']
    filled_df['Pressure9am'] = filled_df['Pressure9am_donor']
    filled_df['Date'] = filled_df.index
    filled_df.reset_index(drop=True, inplace=True)
    
    # save the new variables (they get overwritten)
    collect_df = collect_df.append(filled_df)

In [27]:
# merge pressure values and fill gaps
new_pressure_df = pd.merge(new_pressure_df, collect_df, on=['Date','Location'], how='left')
new_pressure_df['Pressure9am'] = new_pressure_df['Pressure9am_x'].combine_first(new_pressure_df['Pressure9am_y'])
new_pressure_df['Pressure3pm'] = new_pressure_df['Pressure3pm_x'].combine_first(new_pressure_df['Pressure3pm_y'])
new_pressure_df = new_pressure_df[['Date','Location','Pressure9am','Pressure3pm']]

#replace values in main df
df['Pressure9am'] = new_pressure_df['Pressure9am']
df['Pressure3pm'] = new_pressure_df['Pressure3pm']

### 3.3 Temperature Using min and max values

In [28]:
## Adding new feature Min Temp
#Filter to rows where MinTemp is not null and get average temps
not_null_mintemp= df.loc[df['MinTemp'].notnull(), ('MinTemp', 'Location','Month_Num')]
#Average temps
mean_temperatures = not_null_mintemp.groupby (['Location', 'Month_Num']).agg(avg_min_temp = ('MinTemp', 'mean'))

# Join onto original df
min_temp_clean_df = pd.merge (df, mean_temperatures
                                ,how='left'
                                ,on=['Location','Month_Num'])

# Create new test column coalescing original min temp with new one, if null
df['MinTemp'] = min_temp_clean_df.MinTemp.combine_first(min_temp_clean_df.avg_min_temp)

In [29]:
## Adding new feature Max Temp
#Filter to rows where MaxTemp is not null and get average temps
not_null_maxtemp= df.loc[df['MaxTemp'].notnull(), ('MaxTemp', 'Location','Month_Num')]
#Average temps
mean_temperatures = not_null_maxtemp.groupby (['Location', 'Month_Num']).agg(avg_max_temp = ('MaxTemp', 'mean'))
# Join onto original df
max_temp_clean_df = pd.merge (df, mean_temperatures
                                ,how='left'
                                ,on=['Location','Month_Num'])
# Create new test column coalescing original min temp with new one, if null
df['MaxTemp'] = max_temp_clean_df.MaxTemp.combine_first(max_temp_clean_df.avg_max_temp)

In [30]:
## Cleaning Temp 3pm
#Filter to rows where Temp3pm is not null and get average temps
not_null_temp3pm = df.loc[df['Temp3pm'].notnull(), ('Temp3pm', 'Location','Month_Num')]
#Average temps
mean_temperatures = not_null_temp3pm.groupby (['Location', 'Month_Num']).agg(avg_temp3pm = ('Temp3pm', 'mean'))
# Join onto original df
temp3pm_clean_df = pd.merge (df, mean_temperatures
                                ,how='left'
                                ,on=['Location','Month_Num'])
# Create new test column coalescing original min temp with new one, if null
df['Temp3pm'] = temp3pm_clean_df.Temp3pm.combine_first(temp3pm_clean_df.avg_temp3pm)

In [31]:
## Cleaning Temp 9am
#Filter to rows where Temp9pm is not null and get average temps
not_null_temp9am = df.loc[df['Temp9am'].notnull(), ('Temp9am', 'Location','Month_Num')]
#Average temps
mean_temperatures = not_null_temp9am.groupby (['Location', 'Month_Num']).agg(avg_temp9am = ('Temp9am', 'mean'))
# Join onto original df
temp9am_clean_df = pd.merge (df, mean_temperatures
                                ,how='left'
                                ,on=['Location','Month_Num'])
# Create new test column coalescing original min temp with new one, if null
df['Temp9am'] = temp9am_clean_df.Temp9am.combine_first(temp9am_clean_df.avg_temp9am)

### 3.4 Replacing leftover columns with one month average

In [32]:
# Function to take an individual column and replace nulls with month-location averages (or modes)
def month_loc_avg(column, df):
  #Filter to rows where columns is not null
  not_null = df.loc[df[column].notnull(), (column, 'Location','Month_Num')]
  
  
  if is_numeric_dtype(df[column]):
    # Average values
    means = not_null.groupby (['Location', 'Month_Num']).agg(avg_calc = (column, 'mean'))
  else:
    # Modal value
    means = not_null.groupby (['Location', 'Month_Num']).agg(avg_calc = (column, lambda x: scipy.stats.mode(x, nan_policy='omit')[0]))
  # Join onto original df
  clean_df = pd.merge (df, means
                                ,how='left'
                                ,on=['Location','Month_Num'])
  # replace column coalescing original values with new, if null
  df[column] = clean_df[column].combine_first(clean_df['avg_calc'])
  return df

In [33]:
for c in df.columns:
    if df[c].isnull().sum() > 0:
      df = month_loc_avg(c, df)

In [35]:
# Wind gust dir sometimes has entire months of nulls
df['WindGustDir'] = df['WindGustDir'].combine_first(df['WindDir9am'])

### 3.5 Creating Additional Features

In [36]:
# Create some features
rain_yday = df[['Year','DayOfYear','Location','RainToday']]
rain_yday['DayOfYear'] = rain_yday['DayOfYear'] + 1 # won't work for days 365/366
rain_yday['RainYesterday'] = rain_yday['RainToday']
fix_idx = rain_yday['DayOfYear'] > 365
rain_yday[fix_idx].DayOfYear = 1
rain_yday[fix_idx].Year = rain_yday[fix_idx].Year+1
rain_yday.drop('RainToday', axis=1, inplace=True) 
df = pd.merge(df, rain_yday, how='left', on=['Location', 'Year','DayOfYear'])

df['Humidity3pm_x_windspeed'] = df['Humidity3pm'] * df['WindGustSpeed']
df['Sunshine_x_pressure3pm'] = df['Pressure3pm'] * df['Sunshine']
df['HumidityChangeToday'] = df['Humidity3pm'] - df['Humidity9am']
df['PressureChangeToday'] = df['Pressure3pm'] - df['Pressure9am']

df['RainTodayBinary'] = df['RainToday'].replace({"Yes":1,"No":0})

<ipython-input-36-cbf2186764ee>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rain_yday['DayOfYear'] = rain_yday['DayOfYear'] + 1 # won't work for days 365/366
<ipython-input-36-cbf2186764ee>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rain_yday['RainYesterday'] = rain_yday['RainToday']
C:\Users\wblack\Anaconda3\lib\site-packages\pandas\core\generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [37]:
# Forward fill again - should only make minimal changes at this point
df = df.ffill()

### 3.6 Data Cleaning Function

In [41]:
# clean data function wraps up al of the above into one function
def clean_data(df):
  get_predictor = False
  y = 0
  if 'RainTomorrow' in df.columns:
    get_predictor = True
    y = df.pop('RainTomorrow')

  # Convert dates to datetimes
  df['Date'] = pd.to_datetime(df['Date'], format="%d/%m/%Y")
  # Convert location to lat/long
  df['Latitude'] = df['Location'].map(latdict)
  df['Latitude'] = df['Latitude'].astype(float)

  df['Longitude'] = df['Location'].map(longdict)
  df['Longitude'] = df['Longitude'].astype(float)
  # Add some other useful values
  df['Month_Num'] = pd.DatetimeIndex(df['Date']).month
  df['Year'] = pd.DatetimeIndex(df['Date']).year
  df['DayOfYear'] = pd.DatetimeIndex(df['Date']).dayofyear


  # Replaces 'Cloud9am' nulls with linearly regressed sunshine or cloud3pm of humidity3pm values 
  cloud9am = np.array(df['Cloud9am']).reshape(-1, 1) 
  sunshine = np.array(df['Sunshine']).reshape(-1, 1)
  cloud3pm = np.array(df['Cloud3pm']).reshape(-1, 1)
  humidity3pm = np.array(df['Humidity3pm']).reshape(-1, 1)
  for x in range(0, len(cloud9am)):
      if np.isnan(cloud9am[x]):
          if not np.isnan(sunshine[x]):
              cloud9am[x] = sunshine_cloud9.predict(sunshine[x].reshape(-1,1))
          else: 
              if not np.isnan(cloud3pm[x]):
                  cloud9am[x] = cloud3_cloud9.predict(cloud3pm[x].reshape(-1,1))
              else:
                  if not np.isnan(humidity3pm[x]):
                      cloud9am[x] = hum3_cloud9.predict(humidity3pm[x].reshape(-1,1))
                      
  cloud9am_cleaned = cloud9am

  # Replaces 'Cloud3pm' nulls with linearly regressed sunshine or cloud9am or humidity3pm values 
  cloud9am = np.array(df['Cloud9am']).reshape(-1, 1) 
  sunshine = np.array(df['Sunshine']).reshape(-1, 1)
  cloud3pm = np.array(df['Cloud3pm']).reshape(-1, 1)
  humidity3pm = np.array(df['Humidity3pm']).reshape(-1, 1)
  humidity9am = np.array(df['Humidity9am']).reshape(-1, 1)
  for x in range(0, len(cloud3pm)):
      if np.isnan(cloud3pm[x]):
          if not np.isnan(sunshine[x]):
              cloud3pm[x] = sunshine_cloud3.predict(sunshine[x].reshape(-1,1))
          else: 
              if not np.isnan(cloud9am[x]):
                  cloud3pm[x] = cloud9_cloud3.predict(cloud9am[x].reshape(-1,1))
              else:
                  if not np.isnan(humidity3pm[x]):
                      cloud3pm[x] = hum3_cloud3.predict(humidity3pm[x].reshape(-1,1))
                  else:
                      if not np.isnan(humidity9am[x]):
                          cloud9am[x] = hum9_cloud3.predict(humidity9am[x].reshape(-1,1))
                        
                    
  cloud3pm_cleaned = cloud3pm

  # Replaces 'Sunshine' nulls with linearly regressed cloud3pm or cloud9am or humidity3pm or humidity9am values 
  cloud9am = np.array(df['Cloud9am']).reshape(-1, 1) 
  sunshine = np.array(df['Sunshine']).reshape(-1, 1)
  cloud3pm = np.array(df['Cloud3pm']).reshape(-1, 1)
  humidity3pm = np.array(df['Humidity3pm']).reshape(-1, 1)
  humidity9am = np.array(df['Humidity9am']).reshape(-1, 1)
  temp3pm = np.array(df['Temp3pm']).reshape(-1, 1)
  for x in range(0, len(sunshine)):
      if np.isnan(sunshine[x]):
          if not np.isnan(cloud3pm[x]):
              sunshine[x] = cloud3_sunshine.predict(cloud3pm[x].reshape(-1,1))
          else: 
              if not np.isnan(cloud9am[x]):
                  sunshine[x] = cloud9_sunshine.predict(cloud9am[x].reshape(-1,1))
              else:
                  if not np.isnan(humidity3pm[x]):
                      sunshine[x] = hum3_sunshine.predict(humidity3pm[x].reshape(-1,1))
                  else:
                      if not np.isnan(humidity9am[x]):
                          sunshine[x] = hum9_sunshine.predict(humidity9am[x].reshape(-1,1))
                      else:
                          if not np.isnan(temp3pm[x]):
                              sunshine[x] = temp3_sunshine.predict(temp3pm[x].reshape(-1,1))
                          
  sunshine_cleaned = sunshine

  # Replaces 'Evaporation' Nulls with max temp
  evaporation = np.array(df['Evaporation']).reshape(-1, 1)
  maxtemp = np.array(df['MaxTemp']).reshape(-1, 1)
  for x in range(0, len(evaporation)):
      if np.isnan(evaporation[x]):
          if not np.isnan(maxtemp[x]):
              evaporation[x] = maxtemp_evap.predict(maxtemp[x].reshape(-1,1))
          
  evaporation_cleaned = evaporation

  df['Cloud9am'] = cloud9am_cleaned
  df['Cloud3pm'] = cloud3pm_cleaned
  df['Sunshine'] = sunshine_cleaned
  df['Evaporation'] = evaporation_cleaned


  # Take max of AM and PM for newwindgustspeed
  windspeed_df = df[['Date','Location','WindGustSpeed','WindSpeed9am','WindSpeed3pm']]
  windspeed_df['MaxSpeed'] = windspeed_df.max(axis=1)
  df['WindGustSpeed'] = windspeed_df['MaxSpeed']
  
  # Copy pressure values from defined correlated places
  new_pressure_df = df[['Date','Location','Pressure9am','Pressure3pm']]

  pairs = [['MountGinini','Canberra'],
          ['Newcastle','Williamtown'],
          ['Penrith','BadgerysCreek'],
          ['SalmonGums','Walpole']]

  collect_df = pd.DataFrame(columns=new_pressure_df.columns)
  for p in pairs:
    df_missing = new_pressure_df[new_pressure_df['Location']==p[0]]
    df_donor = new_pressure_df[new_pressure_df['Location']==p[1]]
    df_missing.set_index("Date", inplace=True)
    df_donor.set_index("Date", inplace=True)
    filled_df = df_missing.join(df_donor, rsuffix='_donor')
    filled_df['Pressure3pm'] = filled_df['Pressure3pm_donor']
    filled_df['Pressure9am'] = filled_df['Pressure9am_donor']
    filled_df['Date'] = filled_df.index
    filled_df.reset_index(drop=True, inplace=True)
    # save the new variables (they get overwritten)
    collect_df = collect_df.append(filled_df)

  new_pressure_df = pd.merge(new_pressure_df, collect_df, on=['Date','Location'], how='left')
  new_pressure_df['Pressure9am'] = new_pressure_df['Pressure9am_x'].combine_first(new_pressure_df['Pressure9am_y'])
  new_pressure_df['Pressure3pm'] = new_pressure_df['Pressure3pm_x'].combine_first(new_pressure_df['Pressure3pm_y'])
  new_pressure_df = new_pressure_df[['Date','Location','Pressure9am','Pressure3pm']]
  df['Pressure9am'] = new_pressure_df['Pressure9am']
  df['Pressure3pm'] = new_pressure_df['Pressure3pm']

  ## Min Temp
  #Filter to rows where MinTemp is not null and get average temps
  not_null_mintemp= df.loc[df['MinTemp'].notnull(), ('MinTemp', 'Location','Month_Num')]
  #Average temps
  mean_temperatures = not_null_mintemp.groupby (['Location', 'Month_Num']).agg(avg_min_temp = ('MinTemp', 'mean'))

  # Join onto original df
  min_temp_clean_df = pd.merge (df, mean_temperatures
                                ,how='left'
                                ,on=['Location','Month_Num'])

  # Create new test column coalescing original min temp with new one, if null
  df['MinTemp'] = min_temp_clean_df.MinTemp.combine_first(min_temp_clean_df.avg_min_temp)

  ## Max Temp
  #Filter to rows where MaxTemp is not null and get average temps
  not_null_maxtemp= df.loc[df['MaxTemp'].notnull(), ('MaxTemp', 'Location','Month_Num')]
  #Average temps
  mean_temperatures = not_null_maxtemp.groupby (['Location', 'Month_Num']).agg(avg_max_temp = ('MaxTemp', 'mean'))
  # Join onto original df
  max_temp_clean_df = pd.merge (df, mean_temperatures
                                ,how='left'
                                ,on=['Location','Month_Num'])
  # Create new test column coalescing original min temp with new one, if null
  df['MaxTemp'] = max_temp_clean_df.MaxTemp.combine_first(max_temp_clean_df.avg_max_temp)

  ## Temp 3pm
  #Filter to rows where Temp3pm is not null and get average temps
  not_null_temp3pm = df.loc[df['Temp3pm'].notnull(), ('Temp3pm', 'Location','Month_Num')]
  #Average temps
  mean_temperatures = not_null_temp3pm.groupby (['Location', 'Month_Num']).agg(avg_temp3pm = ('Temp3pm', 'mean'))
  # Join onto original df
  temp3pm_clean_df = pd.merge (df, mean_temperatures
                                ,how='left'
                                ,on=['Location','Month_Num'])
  # Create new test column coalescing original min temp with new one, if null
  df['Temp3pm'] = temp3pm_clean_df.Temp3pm.combine_first(temp3pm_clean_df.avg_temp3pm)

  ## Temp 9am
  #Filter to rows where Temp9pm is not null and get average temps
  not_null_temp9am = df.loc[df['Temp9am'].notnull(), ('Temp9am', 'Location','Month_Num')]
  #Average temps
  mean_temperatures = not_null_temp9am.groupby (['Location', 'Month_Num']).agg(avg_temp9am = ('Temp9am', 'mean'))
  # Join onto original df
  temp9am_clean_df = pd.merge (df, mean_temperatures
                                ,how='left'
                                ,on=['Location','Month_Num'])
  # Create new test column coalescing original min temp with new one, if null
  df['Temp9am'] = temp9am_clean_df.Temp9am.combine_first(temp9am_clean_df.avg_temp9am)

  # Fill gaps with month avg
  for c in df.columns:
    if df[c].isnull().sum() > 0:
      df = month_loc_avg(c, df)

  # Wind gust dir must have entire months of nulls
  df['WindGustDir'] = df['WindGustDir'].combine_first(df['WindDir9am'])

  # Create some features
  rain_yday = df[['Year','DayOfYear','Location','RainToday']]
  rain_yday['DayOfYear'] = rain_yday['DayOfYear'] + 1 # won't work for days 365/366
  rain_yday['RainYesterday'] = rain_yday['RainToday']
  fix_idx = rain_yday['DayOfYear'] > 365
  rain_yday[fix_idx].DayOfYear = 1
  rain_yday[fix_idx].Year = rain_yday[fix_idx].Year+1
  rain_yday.drop('RainToday', axis=1, inplace=True) 
  df = pd.merge(df, rain_yday, how='left', on=['Location', 'Year','DayOfYear'])

  df['Humidity3pm_x_windspeed'] = df['Humidity3pm'] * df['WindGustSpeed']
  df['Sunshine_x_pressure3pm'] = df['Pressure3pm'] * df['Sunshine']
  df['HumidityChangeToday'] = df['Humidity3pm'] - df['Humidity9am']
  df['PressureChangeToday'] = df['Pressure3pm'] - df['Pressure9am']

  df['RainTodayBinary'] = df['RainToday'].replace({"Yes":1,"No":0})

  # Forward fill again - should only make minimal changes at this point
  df = df.ffill()

  if get_predictor:
    df['RainTomorrow'] = y
    df = df.bfill()
  return df

## 4 Cleaning Data and Exporting Train Eval Test Split

In [42]:
# loading train set and checking null count
df = pd.read_csv("Kaggle Training Data.csv")
print(df.shape)
df = clean_data(df)
print(df.shape)
df.isnull().sum()

(116367, 23)


<ipython-input-41-e8050777f60a>:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  windspeed_df['MaxSpeed'] = windspeed_df.max(axis=1)
<ipython-input-41-e8050777f60a>:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rain_yday['DayOfYear'] = rain_yday['DayOfYear'] + 1 # won't work for days 365/366
<ipython-input-41-e8050777f60a>:201: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

(116367, 34)


Date                       0
Location                   0
MinTemp                    0
MaxTemp                    0
Rainfall                   0
Evaporation                0
Sunshine                   0
WindGustDir                0
WindGustSpeed              0
WindDir9am                 0
WindDir3pm                 0
WindSpeed9am               0
WindSpeed3pm               0
Humidity9am                0
Humidity3pm                0
Pressure9am                0
Pressure3pm                0
Cloud9am                   0
Cloud3pm                   0
Temp9am                    0
Temp3pm                    0
RainToday                  0
Latitude                   0
Longitude                  0
Month_Num                  0
Year                       0
DayOfYear                  0
RainYesterday              0
Humidity3pm_x_windspeed    0
Sunshine_x_pressure3pm     0
HumidityChangeToday        0
PressureChangeToday        0
RainTodayBinary            0
RainTomorrow               0
dtype: int64

In [43]:
# loading test set and checking null count
test_df = pd.read_csv("Kaggle Test Data.csv")
print(test_df.shape)
df_test = clean_data(test_df)
print(df_test.shape)

(29093, 22)


<ipython-input-41-e8050777f60a>:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  windspeed_df['MaxSpeed'] = windspeed_df.max(axis=1)


(29093, 33)


<ipython-input-41-e8050777f60a>:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rain_yday['DayOfYear'] = rain_yday['DayOfYear'] + 1 # won't work for days 365/366
<ipython-input-41-e8050777f60a>:201: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rain_yday['RainYesterday'] = rain_yday['RainToday']
C:\Users\wblack\Anaconda3\lib\site-packages\pandas\core\generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [46]:
# Assign train/eval split - last 20% of data should be eval
unique_dates = df['Date'].unique()
cutoff = int(len(unique_dates)*0.8)
train_dates = unique_dates[0:cutoff]
df_train = df[df['Date'].isin(train_dates)]
df_eval = df[~df['Date'].isin(train_dates)]

print(f"Train data runs from ", df_train["Date"].min(), " to ", df_train["Date"].max(), ", ", len(df_train["Date"].unique()), " dates.")
print(f"Evaluation data runs from ", df_eval["Date"].min(), " to ", df_eval["Date"].max(), ", ", len(df_eval["Date"].unique()), " dates.")

Train data runs from  2007-11-01 00:00:00  to  2014-04-20 00:00:00 ,  2274  dates.
Evaluation data runs from  2014-04-21 00:00:00  to  2015-11-10 00:00:00 ,  569  dates.


In [47]:
# Output csvs
df_train.to_csv("TrainData.csv")
df_eval.to_csv("EvalData.csv")
df_test.to_csv("CleanedTestData.csv")